#### Import Library

In [ ]:
# library
import torch
import torch.nn.functional as F
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torchvision.models as models
from torch.optim import SGD
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

# parameters
torch.manual_seed(1)
batch_size = 20

#### Read data from images and excel-labels

In [ ]:
# read the images and names
IMAGE_DIR =  'Data/C_img_jpg'
def read_images(image_path=IMAGE_DIR):
    images = []
    images_names = [image for image in os.listdir(image_path) if not image.startswith('.')] 
    for image_name in images_names: 
            img = Image.open (os.path.join(image_path, image_name))
            images.append(img)
    return images,images_names
images,names = read_images()

In [ ]:
# read the label
excel=pd.read_csv('Data/C_all_data_add_20211007_xz.csv', encoding = 'gb2312')
excel = np.array(excel)
excel = excel.tolist()
# Alligning the labels with images. (i.e. the first label corresponds to the first label.)
labels = []
sub_labels = []

for i in range(len(names)):
    for j in range(len(excel)):
        if names[i] == excel[j][0]:
            labels.append(excel[j][1])
            sub_labels.append(excel[j][2:])


#### Split the training and testing dataet

In [ ]:
# Stratified Random Sampling
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)

names = np.array(names)
labels = np.array(labels)
names = names.reshape((names.shape[0],-1))
labels = labels.reshape((labels.shape[0],-1))
data = np.hstack((names,labels)) # hstack:each name corresponds to one label 
for train_index,test_index in split.split(data,data[:,-1]):
    train_set = data[train_index,:]
    test_set = data[test_index,:]

#### Data Preprocessing

In [ ]:
# data loader
train_data_2 = train_set
test_data_2 = test_set
train_loader_2=DataLoader(train_data_2 , batch_size=batch_size ,shuffle=True)
test_loader_2=DataLoader(test_data_2, batch_size=batch_size,shuffle=False )

#### Load the Network

In [ ]:
# model-2 : DNN
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes=7):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self. fc = nn.Linear(hidden_size, 7)


    def forward(self, x):
        out= self.fc1(x)
        out= self.relu(out)
        out= self.dropout(out)
        out= self.fc2(out)
        out= self.relu(out)
        out= self.fc(out)
        return out

device=torch.device('cuda')
model_2=Net(6, 384, 7)
model_2=model_2.to(device)
print(model_2)

#### Train the Network

In [ ]:
# model 2 training
learning_rate = 0.001
num_epoches = 20
criterion=nn.CrossEntropyLoss()
optimizer=SGD(model_2.parameters(),lr=learning_rate)


train_loss_all = []   # store the loss of training set
train_accur_all = []  # store the accuracy of training set
test_loss_all = []    # store the loss of test set
test_accur_all = []   # store the accuracy of test set

# start training
for epoch in range(num_epoches):
    running_loss=0.0
    running_acc=0.0
    model_2.train()
    for i,data in enumerate(train_loader_2,1):
        inf=data[:,0:6].float()
        label=data[:,6]
        inf=Variable(inf)
        label=Variable(label)
        inf=inf.cuda()
        label=label.cuda()
        # forward
        out=model_2(inf)
        loss=criterion(out,label)
        running_loss+=loss.data*label.size(0)
        _,pred=torch.max(out,1)
        num_correct=(pred==label).sum()
        running_acc+=num_correct.data
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print('Train {} epoch, Loss: {:.6f},Acc: {:.6f}'.format(epoch+1,running_loss/(len(train_data_2)),running_acc/len(train_data_2)))
    train_loss_all.append(running_loss / len(train_data_2))   # store the loss of training set, and then plot it
    train_accur_all.append(running_acc/len(train_data_2))     # store the accuracy of training set, and then plot it
    
    model_2.eval()
    with torch.no_grad():
        eval_loss = 0
        eval_acc = 0
        train_loss_results = []

        for data in test_loader_2:
            inf=data[:,0:6].float()
            label=data[:,6]
            inf = Variable(inf)
            inf=inf.cuda()
            label=label.cuda()
            out = model_2(inf)
            loss = criterion(out, label)
            eval_loss += loss.data
            _,pred = torch.max(out,1)
            num_correct = (pred == label).sum()
            eval_acc += num_correct.data

    print('Test Loss: {:,.6f}, Acc: {:,.6f}'.format(eval_loss/(len(test_data_2)), eval_acc/(len(test_data_2))))
    test_loss_all.append(eval_loss/(len(test_data_2)))
    test_accur_all.append(eval_acc/(len(test_data_2)))


#### Result Visualization

In [ ]:
# transfer the data to cpu
for i in range(len(train_loss_all)):
    train_loss_all[i] = torch.Tensor.cpu(train_loss_all[i])
    train_accur_all[i] = torch.Tensor.cpu(train_accur_all[i])
    test_loss_all[i] = torch.Tensor.cpu(test_loss_all[i])
    test_accur_all[i] = torch.Tensor.cpu(test_accur_all[i])

epoches = [i+1 for i in range(num_epoches)]

# plot the loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(2, 2, 1)
plt.plot(epoches, train_loss_all,"ro-", label="Train loss")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()
plt.subplot(2, 2, 2)
plt.plot(epoches, train_accur_all,"ro-", label="Train accur")
plt.xlabel("epoch")
plt.ylabel("acc")
plt.legend()
plt.subplot(2, 2, 3)
plt.plot(epoches, test_loss_all,"bo-", label="Test loss")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()
plt.subplot(2, 2, 4)
plt.plot(epoches, test_accur_all,"bo-", label="Test accur")
plt.xlabel("epoch")
plt.ylabel("acc")
plt.legend()
plt.show()

#### Save the model


In [ ]:
torch.save(model_2, '/Mmodel_2_new.pth')
torch.save(model_2.state_dict(),'model_2_param_new.pkl')